In [1]:
from arcgis.gis import GIS

import time
from datetime import datetime

import getpass

C:\Users\rami8629\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
username = input("Enter your username: ")
password = getpass.getpass("Enter your password: ")

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", username, password)


In [4]:
# Get the License Manager
license_manager = gis.admin.license

# Get the ArcGIS Pro License
pro_license = license_manager.get('ArcGIS Pro')

# Get all the licenses
all_licenses = pro_license.all()

# Get the usernames of users who didn't login in the past 3 months
pro_usernames = [user['username'] for user in all_licenses]

print(pro_usernames)


['AGiron_aid', 'AidDevService', 'aiddev_admin', 'ajenkins_EsriAidDev', 'akim.aiddev', 'amakowicki_aid', 'apfister_EsriAidDev', 'avazquezgbd', 'bladds_EsriAidDev', 'bmccardle_aid', 'boneill.aid', 'bstayer_EsriAidDev', 'cbrigham_aid', 'ckiefer_EsriAidDev', 'ckwon_aid', 'cmain_EsriAidDev', 'cmenzel_EsriAidDev', 'cmesserich_aid', 'croland_EsriAidDev', 'cterborgh_aid', 'dgadsden_aid2', 'dmarsh_aiddev', 'dsmetana_aid', 'ESwenson_EsriAidDev', 'Firas_EsriAidDev', 'jhughes_EsriAidDev', 'jvandeusen_aid', 'kmorrish_dev', 'ksmyth_aid', 'lczerwinski.aiddev', 'lndungo_EsriAidDev', 'lowen_aid', 'ltodd.eadev', 'mburger_EsriAidDev', 'mmusgrave_aid', 'npgo_user_aiddev', 'ocottray_EsriAidDev', 'phornstein_aid', 'ralouta.aiddev', 'rdonihue_EsriAidDev4', 'RLehman_aid', 'rrike_esriaiddev', 'speluso_aid', 'ssawaya_aid', 'sydneythomas_dev', 'tbutcher_EsriAidDev', 'vdesrosier_esriaiddev', 'vlusetti_EsriAidDev', 'ykandula.esriaiddev']


In [5]:
# Get the current timestamp in milliseconds
current_timestamp = int(time.time() * 1000)

# Get the timestamp for 3 months ago
three_months_ago = current_timestamp - (3 * 30 * 24 * 60 * 60 * 1000)

user_last_login = {}

# Iterate over the pro_usernames list
for username in pro_usernames:
    # Get the user
    user = gis.users.get(username)
    
    # If the user exists and didn't login in the past 3 months or never logged in, add their username and last login date to the dictionary
    if user and (user.lastLogin < three_months_ago or user.lastLogin == -1):
        # Convert the lastLogin timestamp to a datetime object if the user has logged in before
        last_login_date = datetime.fromtimestamp(user.lastLogin / 1000) if user.lastLogin != -1 else None
        user_last_login[username] = last_login_date
print(user_last_login)

{'AGiron_aid': datetime.datetime(2023, 8, 23, 19, 5, 55), 'aiddev_admin': datetime.datetime(2015, 6, 25, 15, 6, 57), 'ajenkins_EsriAidDev': datetime.datetime(2023, 11, 22, 18, 26, 19), 'amakowicki_aid': datetime.datetime(2022, 8, 2, 20, 50, 11), 'bladds_EsriAidDev': datetime.datetime(2019, 12, 11, 16, 20, 2), 'bmccardle_aid': datetime.datetime(2021, 10, 1, 16, 0, 54), 'boneill.aid': datetime.datetime(2023, 10, 2, 16, 55, 57), 'bstayer_EsriAidDev': datetime.datetime(2023, 11, 2, 14, 2, 37), 'cbrigham_aid': datetime.datetime(2024, 1, 8, 15, 23, 14), 'ckiefer_EsriAidDev': datetime.datetime(2020, 7, 29, 15, 43, 4), 'cmain_EsriAidDev': datetime.datetime(2020, 1, 2, 17, 11, 1), 'cmenzel_EsriAidDev': datetime.datetime(2020, 12, 3, 2, 54, 41), 'cmesserich_aid': datetime.datetime(2023, 9, 22, 16, 59, 12), 'croland_EsriAidDev': datetime.datetime(2022, 9, 15, 17, 58, 15), 'dgadsden_aid2': datetime.datetime(2022, 12, 1, 17, 35, 11), 'dsmetana_aid': datetime.datetime(2023, 1, 9, 17, 58, 31), 'Firas

In [6]:
non_publishing_usernames = []

# Iterate over the pro_usernames list
for username in pro_usernames:
    user = gis.users.get(username)
    
    # Get the user's folders
    folders = user.folders
    folders.append(None)  # Add None to check the root folder
    
    # Initialize a flag to indicate whether the user has published any items in the past 3 months
    has_published_recently = False
    
    # Iterate over the folders
    for folder in folders:
        # Get the items in the folder
        items = user.items(folder=folder, max_items=100)
        
        # Check if any item was created in the past 3 months
        if any(item['created'] > three_months_ago for item in items):
            has_published_recently = True
            break
    
    # If the user hasn't published any items in the past 3 months, add their username to the list
    if not has_published_recently:
        non_publishing_usernames.append(username)

print(non_publishing_usernames)